In [3]:
from google.colab import drive
drive.mount("/content/drive")
%cd drive/MyDrive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: 'drive/MyDrive/'
/content/drive/MyDrive


In [4]:
!python --version
!pip install pandas
!pip install geopy
!pip install numpy
!pip install gurobipy




Python 3.10.6


In [5]:
import pandas as pd


def load_total_demand():
  yearly_sales = pd.read_csv('vw_yearly_sales.csv')
  return yearly_sales


def load_dcs():
  # 2 samples
  # dcs= pd.read_csv('small_list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])

  # FULL DATASET
  dcs = pd.read_csv('list_of_dcs.csv', encoding= 'unicode_escape', names=['ID', "Name","City","State","Address Line 1","Address Line 2", "Coordinates", "Area m3", "Facility Type", "StateConst"])
  #stateconst is a constant used to account for the different prices for the cost of living and rent in various states

  dcs['Area sqft'] = round(dcs['Area m3'] * 10.7639)
  return dcs



def load_dealers():
  # https://www.vw.com/app/dccsearch/vw-us/en/Find%20a%20Volkswagen%20Dealer/+/38.353354499999995/-95.3817145/3/+/+/+/+

  # 2 SAMPLES
  # dealers= pd.read_csv('small_list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])

  # FULL DATASET
  dealers= pd.read_csv('list_of_dealers.csv',names=['ID', "Name","Address Line 1","Address Line 2", "Website","Phone Number", "Coordinates"])
  return dealers


def load_plants():
  plants= pd.read_csv('list_of_vw_plants_and_products.csv', encoding= 'unicode_escape',names=['ID', "City","Country", "Model","Coordinates","Current/Former plant"])

  plants_usa = plants.loc[plants["Country"]=="USA"]

  return plants_usa

def load_dealer_demands():

  def single_sample():

  # 4 samples
    dealers= pd.read_csv('small_list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na"])
    del dealers[dealers.columns[-1]]
    return dealers

  def full_data():
  # FULL DATASET
    dealers= pd.read_csv('list_of_demand_data.csv',names=['ID',"Name","Rating","Proportion","na", "na2", "na3"])
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    del dealers[dealers.columns[-1]]
    return dealers

  dealers = full_data() # either call single_sample or full_data

  return dealers


In [6]:

load_dealer_demands() # used google ratings as a indicator of demand/traffic


,ID,Name,Rating,Proportion
0,0,Alexandria Volkswagen,4.4,2984
1,1,Ancira Volkswagen,4.5,1913
2,2,Ancira Volkswagen of Laredo,4.1,271
3,3,Andy Mohr Volkswagen of Avon,4.5,1432
4,4,Antelope Valley Volkswagen,4.2,458
...,...,...,...,...
633,633,Wynn Volkswagen,4.6,432
634,634,"York Volkswagen, Inc.",4.3,380
635,635,Young Volkswagen of Layton,4.4,1153
636,636,Zimbrick Volkswagen,4.7,633


In [7]:
load_total_demand()

,year,ID4,TAOS,Cross Sport,Arteon,Tiguan,Atlas,Routan,Touareg,Tiguan.1,CC,EOS,Passat,Jetta,Golf,Beetle
0,2008,0,0,0,0,0,0,3387,6755,8664,2105,12837,30034,97461,35408,26477
1,2009,0,0,0,0,0,0,14651,4392,13903,23872,7204,11138,108427,15752,14085
2,2010,0,0,0,0,0,0,15961,4713,20946,27987,6690,12497,123213,28286,16573
3,2011,0,0,0,0,0,0,12473,7535,2599,29502,7533,22835,177360,34706,6468
4,2012,0,0,0,0,0,0,10484,10553,31731,21646,6214,117023,170424,40885,29174
5,2013,0,0,0,0,0,0,2109,8233,3000,15672,4178,109652,163793,30931,43134
6,2014,0,0,0,0,0,0,1103,6961,25121,9995,3411,96649,160873,33675,29182
7,2015,0,0,0,0,0,0,0,8223,35843,6276,2993,78207,131109,65308,22667
8,2016,0,0,0,0,0,0,0,4223,43638,3237,387,73002,121107,61687,15667
9,2017,0,0,0,0,21023,27119,0,3545,25960,1355,1,60722,115807,68978,15166


In [8]:
load_dcs()

,ID,Name,City,State,Address Line 1,Address Line 2,Coordinates,Area m3,Facility Type,StateConst,Area sqft
0,0,Volkswagen Parts Distribution Center,Ontario,CA,3095 E Cedar St,"Ontario, CA 91761","34.0376080290585, -117.58641507359258",30456,1,511,327825.0
1,1,Volkswagen Parts Distribution Center,Rocklin,CA,1100 Tinker Rd,"Rocklin, CA 95765, United States","38.81328171265908, -121.30150120229099",18360,1,511,197625.0
2,2,Volkswagen Parts Distribution Center,Jacksonville,FL,4150 Perimeter Industrial Pkwy N,"Jacksonville, FL 32219, United States","30.373771993405768, -81.75890718718041",362670,2,447,3903744.0
3,3,Volkswagen Parts Distribution Center,Cranbury,NJ,47 Station Rd,"East Windsor, NJ 08512, United States","40.3071068545627, -74.50867073108051",103200,2,519,1110834.0
4,4,Volkswagen Regional Distribution Center,Lenoir City,TN,2560 Buttermilk Rd W,"Lenoir City, TN 37771, United States","35.88214676007374, -84.3612681600479",45000,1,411,484376.0
5,5,Volkswagen Group of America Parts Distribution...,Ft. Worth,TX,230 Intermodal Pkwy,"Fort Worth, TX 76177, United States","32.96678488424622, -97.32814574044846",55954,1,439,602283.0
6,6,Volkswagen Parts Distribution Center,Pleasant Prairie,WI,11589 88th Ave,"Pleasant Prairie, WI 53158, United States","42.5108270307348, -87.91110551996559",23217,1,419,249905.0


In [9]:
load_dealers()

,ID,Name,Address Line 1,Address Line 2,Website,Phone Number,Coordinates
0,0,Alexandria Volkswagen,107 W Glebe Rd,"Alexandria, VA 22305",https://alexandriavw.com,(703) 684-8888,"38.83578067871878, -77.05799331320739"
1,1,Ancira Volkswagen,6125 Bandera Rd,"San Antonio, TX 78238",https://anciravolkswagen.com,(210) 681-2300,"29.488410637739907, -98.60618875952487"
2,2,Ancira Volkswagen of Laredo,2701 N.E. Bob Bullock Loop,"Laredo, TX 78045",https://vwlaredo.com,(956) 717-2200,"27.612614635541124, -99.47040125767218"
3,3,Andy Mohr Volkswagen of Avon,8791 E Us Highway 36,"Avon, IN 46123",https://www.andymohrvw.com,(317) 978-0296,"39.76232468061578, -86.36526215581948"
4,4,Antelope Valley Volkswagen,39050 Carriage Way,"Palmdale, CA 93551",https://vwantelopevalley.com,(661) 273-2500,"39.764347438993674, -86.37612026718733"
...,...,...,...,...,...,...,...
633,633,Wynn Volkswagen,2021 W Main St,"Norristown, PA 19403",https://jimwynnvw.com,(610) 539-1100,"40.133512710158975, -75.37891900773616"
634,634,"York Volkswagen, Inc.",3475 E Market St,"York, PA 17402",https://www.yorkvw.com,(717) 755-1015,"39.98167762462809, -76.65493938869697"
635,635,Young Volkswagen of Layton,1230 N Main St,"Layton, UT 84041",https://www.youngvolkswagen.com,(801) 544-5878,"41.07908703469315, -111.98269918866244"
636,636,Zimbrick Volkswagen,7505 Century Ave,"Middleton, WI 53562",https://www.z-vw.com,(608) 836-7777,"43.11080320619693, -89.51033578674627"


In [10]:
load_plants()

,ID,City,Country,Model,Coordinates,Current/Former plant
25,9,Chattanooga,USA,Atlas,"35.077103086458976, -85.13084428224508",current
26,9,Chattanooga,USA,Cross Sport,"35.077103086458976, -85.13084428224508",current
27,9,Chattanooga,USA,ID4,"35.077103086458976, -85.13084428224508",current
28,9,Chattanooga,USA,Passat,"35.077103086458976, -85.13084428224508",former


In [11]:
### do location work
import networkx as nx
from geopy.distance import geodesic
import numpy

# ALL DISTANCES ARE IN MILES!!!!
# ALL AREAS ARE IN SQFT!!!
def dc_to_dealer_distances(dcs, dealers): # using geodesic distance, may want to add 15% or something onto distance
  distance_dc_to_dealer = numpy.zeros((len(dcs), len(dealers)))
  for dc_id, dc in dcs.iterrows():
    for dealer_id, dealer in dealers.iterrows():
      point1 = dc["Coordinates"].split(",")
      point2 = dealer["Coordinates"].split(",")
      distance = geodesic(point1, point2).miles
      dc_id = dc["ID"]
      dealer_id = dealer["ID"]
      # print ("distance from dc " + str(dc_id)+ " to dealer "+ str(dealer_id) + " is " + str(distance))
      distance_dc_to_dealer[dc_id][dealer_id] = distance
  distance_dc_to_dealer_df = pd.DataFrame(distance_dc_to_dealer)
  return distance_dc_to_dealer_df


def plant_to_dc_distances(plants, dcs):
  # get coordinates of the 1 usa plant
  plant_coordinates = plants['Coordinates'].iloc[0].split(",")
  distance_plants_to_dcs = [] # since we only have 1 usa plant we can just make everything from there (very convenient)
  for dcs_id, dc in dcs.iterrows():
    point1 = plant_coordinates
    point2 = dc["Coordinates"].split(",")
    distance = geodesic(point1, point2).miles
    dc_id = dc["ID"]
    distance_plants_to_dcs.append(distance)
  return distance_plants_to_dcs

plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)


print(plants_to_dcs)
print(dcs_to_dealers)

[1844.0657073866873, 2005.3600623066698, 378.90622778556326, 684.0783183176076, 70.45212920438283, 714.4625319935964, 534.2000359433654]
           0            1            2            3            4    \
0  2263.452155  1158.436165  1162.247798  1764.014177  1763.444191   
1  2363.262580  1445.165292  1475.686061  1862.087265  1861.494162   
2   641.631502  1011.662965  1087.963110   697.606938   697.964214   
3   169.780411  1551.252779  1670.501239   629.457738   630.013730   
4   450.488983   938.386559  1054.381785   289.172004   289.525934   
5  1203.759603   251.291467   390.384571   769.254301   768.854000   
6   623.412092  1077.416837  1215.562809   206.090780   205.740888   

           5            6            7            8            9    ...  \
0  2287.066337  1327.394925   826.190056  2394.271290  1569.201382  ...   
1  2382.774699  1595.941164   457.841986  2483.978200  1688.186791  ...   
2   670.952648   827.865628  2427.686953   743.500442   731.714001  ...   
3 

In [14]:
plant_to_dc_distances(plants, dcs)

[1844.0657073866873,
 2005.3600623066698,
 378.90622778556326,
 684.0783183176076,
 70.45212920438283,
 714.4625319935964,
 534.2000359433654]

In [15]:
## Calculate demands
def get_dealer_demands(year):
  dealers = load_dealers()
  dealer_demands = load_dealer_demands()
  total_demands = load_total_demand() # demand for each specific car type


  def get_demands(year):
    year_demands = total_demands.loc[total_demands['year'] == year]
    year_demands = year_demands[["Atlas","Cross Sport","ID4","Passat"]]
    return year_demands

  year_demands = get_demands(2021)
  year_demands = year_demands.values.tolist()[0]

  # print(year_demands)

  proportion_sum = dealer_demands["Proportion"].sum()

  dealer_demands [["Ratio of sales"]] = None
  dealer_demands["Ratio of sales"] = dealer_demands["Proportion"]/proportion_sum

  # print(dealer_demands)

  # # with the sales proportion, get the general sales for each location

  temp_ratio_col = pd.DataFrame(dealer_demands["Ratio of sales"])

  temp = []
  for x in list(dealer_demands["Ratio of sales"]):
    temp1=[]
    for y in list(year_demands):
      temp1.append(round(x*y))
    temp.append(temp1)
  sales = pd.DataFrame(temp, columns=["Atlas","Cross Sport","ID4","Passat"])
  return sales

In [ ]:
sales = get_dealer_demands(2021)
sales
# the left side are the dealer ids

,Atlas,Cross Sport,ID4,Passat
0,405,242,94,137
1,260,155,60,88
2,37,22,9,12
3,194,116,45,66
4,62,37,14,21
...,...,...,...,...
633,59,35,14,20
634,52,31,12,17
635,157,94,36,53
636,86,51,20,29


In [16]:
## Insert something here with defining facilities


def calculate_dc_cost():


  def calculate_dc_cost_monthly(id, state, facility_type, areasqft):
  # print (id, facility, state, areasqft)
    cost = 1.5*areasqft #2$ baseline
    insurance = 3850  # https://glcdistribution.com/customer-resources/tools/warehousing-insurance-calculator/ 1 million dollar standard

    employee_salaries = (areasqft/2000)*20*(40*4) # https://www.payscale.com/research/CA/Job=Warehouse_Material_Handler/Hourly_Rate

  # pay 1 dollar per sqft for utility
    utilities = areasqft


  # $600 – $950 per month to lease a $50,000 forklift.
  # https://discord.com/channels/260272353118912522/1093655130076893284/1129635416409133076 1 forklift services 5000 sqft
    forklift_cost =  areasqft/5000  *800


    # https://discord.com/channels/260272353118912522/1093655130076893284/1129639090942398504
    price_it_costs_to_build_warehouse_sqft = 100*areasqft
    property_tax = 6/1000*price_it_costs_to_build_warehouse_sqft


    security = 1000 + (5 * 8 * 28*20) # 1000 fixed cost monitoring + 20$/h * 40h/week * 28 people

    if facility_type == 1:
      ft = 1
    else:
      ft = 1.2

    price = (cost + insurance + employee_salaries + utilities + forklift_cost + security + property_tax)* (state/508)*ft
    # 508 is the price of 100 big macs and state is the price of 100 big macs in that state
    return price


  prices = []

  for i in range(len(dcs.index)):
      moneys = calculate_dc_cost_monthly(float(dcs.loc[i,["ID"]]), float(dcs.loc[i,["StateConst"]]), float(dcs.loc[i,["Facility Type"]]),float(dcs.loc[i,["Area sqft"]] ))
      prices.append(moneys*12)
  return prices
  # print(calculate_dc_cost())


# C_i
def calculate_dc_shipment_capacity():
  dcs = load_dcs()
  dcs = dcs[["Area sqft",'Facility Type']]


# The largest car to consider is the Cross Sport
# It has dimensions: 4,966 mm L x 1,990 mm W x 1,723 mm H
# we can round this up to: 17 ft length, 7 foot wide, 6 foot tall
# add about 2 ft to everything as a buffer
# new measurement is about 18 ft length, 8ft wide, 8 foot tall
# so, we can predict that 2 car takes up 144/2, each car takes 72 sq foot

# from https://discord.com/channels/260272353118912522/1093655130076893284/1131271495004459018,
# we see that 60-80% of DCs are for storage. Let's assume 60%


  vehicle_capacity = dcs["Area sqft"]*0.6 /72


# from https://discord.com/channels/260272353118912522/1093655130076893284/1131266537869820075
# we see that most vehicles stay in the dc a couple of weeks (take 2 weeks)
# we can frame this as: All vehicles in the DCS arrive at time 0, and after 2 weeks all are released

  annual_throughput = round(vehicle_capacity) * (52/2)

  # dcs["Annual Throughput"] = annual_throughput

  # print(annual_throughput)
  return (list(annual_throughput))

# WE NEED TO BE CAREFUL ABOUT TYPE 2 FACILITIES:

# print(calculate_dc_cost())

# print(calculate_dc_shipment_capacity())

# Important Verbiage

Has most of the answers:

def generate_facility_cost

---
### Generally 2-3$ per square foot
### https://www.warehousingandfulfillment.com/resources/fulfillment-cost-calculator/

1. Rent or Mortgage:
[A 50,000 square foot steel warehouse, in the words of one construction expert, will cost between 52,800 and 312,500 per month in addition to monthly rent, utilities, insurance, and other associated costs. A 100,000 increase in the monthly budget will be made up of the cost of a turnkey warehouse, which will also be included in the estimate.](https://www.hechtgroup.com/the-average-size-of-a-small-warehouse-is-1-000-5-000-square-feet/)


2. Insurance:
- a standard warehouse general liability is 1-2 million of coverage, which is about 100/month
3. Salaries:
```
There is no definitive answer to how many employees per square foot warehouse as it depends on a variety of factors such as the type of products being stored, the level of automation in the warehouse, and the workflows. However, a good rule of thumb is to have one employee for every 1,000-1,500 square feet of warehouse space. This allows for efficient operations and ensures that there is enough staff to handle the volume of work.Every employee in a warehouse is allocated about 200-250 square feet of space. Distribution centers had a median square footage of 240,410 in 2016, 176,600 in 2017, and 305,000 in 2018. If a warehouse is properly designed, it will be more efficient and require less space per employee than if it is poorly designed. The number of warehouse employees varies greatly. Warehouses in general are less than 10,000 square feet in size. Eight-hour shifts pay between $14.80 and $19.90 per hour on average. Approximately 35% of the total warehouse space is estimated to be occupied by this type of warehouse.
```
`https://www.hechtgroup.com/how-many-employees-should-a-warehouse-have-per-square-foot/`

4. Utilities:
- [average utlity cost per square foot commercial property is about 1 dollar/sqft for electricity](https://www.iotacommunications.com/blog/average-utility-cost-per-square-foot-commercial-property/)
5. Equipment:
875$ lease a forklift

6. Licensing and Permits:

7. Maintenance and Repairs:

8. Property Taxes:

9. Security:

10. Technology:




# Back to Code

In [17]:
## Determining the cost of transportation


## THE FUNCTION OF SHIPPING TRUCKS CAN BE APPROXIMATED BY A PIECEWISE FUNCTION
# https://www.desmos.com/calculator/ipxbog2uyr
# constructed from the data here: https://www.marketwatch.com/guides/car-shipping/truck-shipping/#:~:text=According%20to%20quotes%20we%20received,when%20using%20an%20enclosed%20carrier.

def fixed_cost_shipment(): #T_1
  return 186 # cost to drive 0 miles

def calculate_variable_cost_shipment(mileage): #T_2
  # if mileage < 1135:
  #   return mileage * (551/535)
  # return mileage * (0.43)
  return (0.43)

  # TODO: WE ARE ASSUMING FOR SIMPLICYT SAKE 0.43$/MILE AND ARE NOT CONSIDERING THE OTHER FUNCTION FOR MODELLING RIGHT NOW






In [18]:
## The modelling?
import gurobipy as gp
from gurobipy import GRB


# Create environment with WLS license
e = gp.Env(empty=True)
e.setParam('WLSACCESSID', '47fa1c45-5377-4f93-87d4-3da7da2b6955')
e.setParam('WLSSECRET', '7dfbddcb-7333-4ec7-927d-b459cffd9565')
e.setParam('LICENSEID', 868415)
e.start()


##LOAD DATA

# plants = load_plants()
# dcs = load_dcs()
# dealers = load_dealers()

# dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
# plants_to_dcs = plant_to_dc_distances(plants, dcs)
# dealer_demands = get_dealer_demands(2021)
# dc_annual_cost = calculate_dc_cost()





plants = load_plants()
dcs = load_dcs()
dealers = load_dealers()

dcs_to_dealers = dc_to_dealer_distances(dcs, dealers)
plants_to_dcs = plant_to_dc_distances(plants, dcs)
dealer_demands = get_dealer_demands(2021) ## car demand for each dealership
dc_annual_cost = calculate_dc_cost()



# define sets
plants_set = [0] # set S
dcs_set = list(dcs["ID"]) # set I
dealers_set = list(dealers["ID"]) # set J
vehicle_set = [0,1,2,3] # we define 0 to 3, corresponding to Atlas, CrossSport, ID4 and Passat vehicles (set V)
facility_type_set = [0, 1] # set P

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 868415
Academic license - for non-commercial use only - registered to anita.yang@uwaterloo.ca


In [19]:
dcs_set

[0, 1, 2, 3, 4, 5, 6]

In [20]:
# Parameter prep - we change the dataframes to arrays of sorts, which we can use indexes and such in the formulation

d_jv = dealer_demands.values
# d_jv[1][0]

m_ij = dcs_to_dealers.values

# dc 1 to dealer 2 should be 1475.6
# m_ij[1][2]

c_si = plants_to_dcs

L_j = 1 # load factor is 1

T_1 = fixed_cost_shipment()

T_2 = calculate_variable_cost_shipment("dummy_value") # see code comments on why we do this as a temp hack for a preliminary model

f_ip = dc_annual_cost

C_i = calculate_dc_shipment_capacity()

In [21]:
d_jv

array([[405, 242,  94, 137],
       [260, 155,  60,  88],
       [ 37,  22,   9,  12],
       ...,
       [157,  94,  36,  53],
       [ 86,  51,  20,  29],
       [144,  86,  33,  48]])

In [22]:
m_ij

array([[2263.45215475, 1158.43616471, 1162.24779847, ...,  574.55747332,
        1634.20122788, 1643.46515477],
       [2363.26257953, 1445.1652925 , 1475.68606054, ...,  518.64965043,
        1678.77280951, 1688.2086155 ],
       [ 641.63150249, 1011.66296482, 1087.96311007, ..., 1840.38843611,
         976.88499167,  972.67464451],
       ...,
       [ 450.48898284,  938.38655906, 1054.38178519, ..., 1532.90608227,
         569.30379263,  564.81784318],
       [1203.75960336,  251.29146676,  390.38457119, ...,  982.46417821,
         818.47126211,  824.01117699],
       [ 623.4120919 , 1077.41683659, 1215.56280894, ..., 1242.92807047,
          91.21774722,   83.01822265]])

In [23]:
c_si

[1844.0657073866873,
 2005.3600623066698,
 378.90622778556326,
 684.0783183176076,
 70.45212920438283,
 714.4625319935964,
 534.2000359433654]

In [24]:
L_j

1

In [25]:
T_1

186

In [26]:
T_2

0.43

In [27]:
f_ip

[19560591.129921257,
 11922485.01968504,
 240739435.00573227,
 79824863.09933859,
 23119403.97543307,
 30636794.21622047,
 12290770.811811024]

In [28]:
C_i

[71032.0, 42822.0, 845806.0, 240682.0, 104936.0, 130494.0, 54158.0]

In [29]:
m = gp.Model(env=e)

# CREATE THE DECISION VARIABLES HERE
# : Annual shipment of type v∈V vehicles from source s∈S to DC location i∈I
x1_siv = m.addVars(plants_set, dcs_set, vehicle_set, vtype=GRB.INTEGER) #
x2_ijv = m.addVars(dcs_set, dealers_set, vehicle_set, vtype=GRB.INTEGER )
y_ip = m.addVars(dcs_set, facility_type_set, vtype=GRB.BINARY)

In [30]:
x1_siv

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 3): <gurobi.Var *Awaiting Model Update*>,


In [31]:
x2_ijv

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 4, 3): <gurobi.Var *Awaiting Model Update*>,


In [32]:
y_ip

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 0): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 0): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (5, 0): <gurobi.Var *Awaiting Model Update*>,
 (5, 1): <gurobi.Var *Awaiting Model Update*>,
 (6, 0): <gurobi.Var *Awaiting Model Update*>,
 (6, 1): <gurobi.Var *Awaiting Model Update*>}

In [33]:
# TODO: FIX THIS
#  m.setObjective(, gp.GRB.MINIMIZE)

# obj2_sub1 = sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set for i in dcs_set)
# obj2 = sum(obj2_sub1 * c_si[i] for s in plants_set for i in dcs_set)

# Need to verify them all with Affleck
obj1 = sum((x2_ijv[i, j, v]/L_j)*(T_1 + T_1*m_ij[i, j]) for v in vehicle_set for j in dealers_set for i in dcs_set)

# Since we only have one plant, c_si should only take in i as an index. We are good.
obj2 = sum(sum(x2_ijv[i, j, v] + x1_siv[s, i, v] for v in vehicle_set for j in dealers_set for s in plants_set) * c_si[i] for i in dcs_set)

obj3 = sum(x1_siv[s, i, v] for v in vehicle_set for s in plants_set for i in dcs_set)

# This works for a small example, but not for a larger example
# Can we just do one type of facility?
obj4 = sum(f_ip[i]*y_ip[i, p] for p in facility_type_set for i in dcs_set)

m.setObjective(obj1 + obj2 + obj3 + obj4, gp.GRB.MINIMIZE)
#m.update()
#m.display()

In [34]:
# Constraints:

# Constraint 1: Demand at each market area for each vehicle type must be met
m.addConstrs(sum(x2_ijv[i, j, v] for i in dcs_set) == d_jv[j, v] for v in vehicle_set for j in dealers_set)

# Constraint 2: Vehicle flows between plants and DCs must be conserved
# Ask Affleck about summing over i twice

m.addConstrs(sum(x1_siv[s, i, v] for s in plants_set) == sum(x2_ijv[i, j, v] for j in dealers_set) for i in dcs_set for v in vehicle_set)

# Constraint 3: Total vehicle flow to each DC must satisfy the minimum capacity requirement
m.addConstrs(
    (C_i[i] * y_ip[i, 1] <= sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) for i in dcs_set),
    name="constraint_3_min"
)

# Constraint 3: Total vehicle flow to each DC must satisfy the maximum capacity requirement
m.addConstrs(
    (sum(x2_ijv[i, j, v] for v in vehicle_set for j in dealers_set) <= C_i[i] * y_ip[i, 0] + sum(d_jv[j, v] for v in vehicle_set for j in dealers_set) * y_ip[i, 1] for i in dcs_set),
    name="constraint_3_max"
)

# # Constraint 4: Shipment quantities must be nonnegative
m.addConstrs(x1_siv[s, i, v] >= 0 for s in plants_set for v in vehicle_set for i in dcs_set)
m.addConstrs(x2_ijv[i, j, v] >= 0 for i in dcs_set for v in vehicle_set for j in dealers_set)

# Constraint 6: Ask Affleck
r=60

m.addConstrs((sum(x2_ijv[i, j, v] for v in vehicle_set) == 0 for i in dcs_set for j in dealers_set if m_ij[i][j] / 300 > r)
)


{}

In [35]:
m.optimize()
# m.write("out.sol")

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (linux64)

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Academic license - for non-commercial use only - registered to anita.yang@uwaterloo.ca
Optimize a model with 20486 rows, 17906 columns and 89397 nonzeros
Model fingerprint: 0x6b576eb3
Variable types: 0 continuous, 17906 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+05]
  Objective range  [2e+03, 2e+08]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+02]
Found heuristic solution: objective 1.214071e+11
Presolve removed 17924 rows and 44 columns
Presolve time: 0.05s
Presolved: 2562 rows, 17862 columns, 48467 nonzeros
Variable types: 0 continuous, 17862 integer (89 binary)
Found heuristic solution: objective 1.206215e+11

Root relaxation: objective 3.127777e+10, 2551 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Obje

In [ ]:
# m.display()

In [36]:

# PRINT VALUES OF THE DV HERE:


x1_siv


{(0, 0, 0): <gurobi.Var C0 (value 0.0)>,
 (0, 0, 1): <gurobi.Var C1 (value 0.0)>,
 (0, 0, 2): <gurobi.Var C2 (value 0.0)>,
 (0, 0, 3): <gurobi.Var C3 (value 0.0)>,
 (0, 1, 0): <gurobi.Var C4 (value 0.0)>,
 (0, 1, 1): <gurobi.Var C5 (value 0.0)>,
 (0, 1, 2): <gurobi.Var C6 (value 0.0)>,
 (0, 1, 3): <gurobi.Var C7 (value 0.0)>,
 (0, 2, 0): <gurobi.Var C8 (value 0.0)>,
 (0, 2, 1): <gurobi.Var C9 (value 0.0)>,
 (0, 2, 2): <gurobi.Var C10 (value 0.0)>,
 (0, 2, 3): <gurobi.Var C11 (value 0.0)>,
 (0, 3, 0): <gurobi.Var C12 (value 0.0)>,
 (0, 3, 1): <gurobi.Var C13 (value 0.0)>,
 (0, 3, 2): <gurobi.Var C14 (value 0.0)>,
 (0, 3, 3): <gurobi.Var C15 (value 0.0)>,
 (0, 4, 0): <gurobi.Var C16 (value 72386.0)>,
 (0, 4, 1): <gurobi.Var C17 (value 43306.0)>,
 (0, 4, 2): <gurobi.Var C18 (value 16746.0)>,
 (0, 4, 3): <gurobi.Var C19 (value 24392.0)>,
 (0, 5, 0): <gurobi.Var C20 (value 0.0)>,
 (0, 5, 1): <gurobi.Var C21 (value 0.0)>,
 (0, 5, 2): <gurobi.Var C22 (value 0.0)>,
 (0, 5, 3): <gurobi.Var C23 

In [37]:
x2_ijv

{(0, 0, 0): <gurobi.Var C28 (value -0.0)>,
 (0, 0, 1): <gurobi.Var C29 (value -0.0)>,
 (0, 0, 2): <gurobi.Var C30 (value -0.0)>,
 (0, 0, 3): <gurobi.Var C31 (value -0.0)>,
 (0, 1, 0): <gurobi.Var C32 (value -0.0)>,
 (0, 1, 1): <gurobi.Var C33 (value -0.0)>,
 (0, 1, 2): <gurobi.Var C34 (value -0.0)>,
 (0, 1, 3): <gurobi.Var C35 (value -0.0)>,
 (0, 2, 0): <gurobi.Var C36 (value -0.0)>,
 (0, 2, 1): <gurobi.Var C37 (value -0.0)>,
 (0, 2, 2): <gurobi.Var C38 (value -0.0)>,
 (0, 2, 3): <gurobi.Var C39 (value -0.0)>,
 (0, 3, 0): <gurobi.Var C40 (value -0.0)>,
 (0, 3, 1): <gurobi.Var C41 (value -0.0)>,
 (0, 3, 2): <gurobi.Var C42 (value -0.0)>,
 (0, 3, 3): <gurobi.Var C43 (value -0.0)>,
 (0, 4, 0): <gurobi.Var C44 (value -0.0)>,
 (0, 4, 1): <gurobi.Var C45 (value -0.0)>,
 (0, 4, 2): <gurobi.Var C46 (value -0.0)>,
 (0, 4, 3): <gurobi.Var C47 (value -0.0)>,
 (0, 5, 0): <gurobi.Var C48 (value -0.0)>,
 (0, 5, 1): <gurobi.Var C49 (value -0.0)>,
 (0, 5, 2): <gurobi.Var C50 (value -0.0)>,
 (0, 5, 3):

In [38]:
y_ip

{(0, 0): <gurobi.Var C17892 (value -0.0)>,
 (0, 1): <gurobi.Var C17893 (value -0.0)>,
 (1, 0): <gurobi.Var C17894 (value -0.0)>,
 (1, 1): <gurobi.Var C17895 (value -0.0)>,
 (2, 0): <gurobi.Var C17896 (value -0.0)>,
 (2, 1): <gurobi.Var C17897 (value 0.0)>,
 (3, 0): <gurobi.Var C17898 (value -0.0)>,
 (3, 1): <gurobi.Var C17899 (value 0.0)>,
 (4, 0): <gurobi.Var C17900 (value -0.0)>,
 (4, 1): <gurobi.Var C17901 (value 1.0)>,
 (5, 0): <gurobi.Var C17902 (value -0.0)>,
 (5, 1): <gurobi.Var C17903 (value -0.0)>,
 (6, 0): <gurobi.Var C17904 (value -0.0)>,
 (6, 1): <gurobi.Var C17905 (value -0.0)>}

In [ ]:
# Also, if there is no DC installed at a location, there is no shipment to that location

In [ ]:
# The distribution centers must be selected so that all market areas can be reached within r days. Suppose a truck travels on average 300 miles a day.